In [14]:
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [15]:
G = np.array([[1, 0.1, 0.2, 0.1, 0],\
[0.1, 1, 0.1, 0.1, 0],\
[0.2, 0.1, 2, 0.2, 0.2],\
[0.1, 0.1, 0.2, 1, 0.1],\
[0, 0, 0.2, 0.1, 1] ])

n = 5
# two groups {1,2} , {3,4,5}

p_max = 3
p_g_1 = 4
p_g_2 = 6
p_rc = 5

sigma = 0.5

In [16]:
p = cp.Variable(n)

# constraints
constraints = [ p>=0 ]
constraints.append( p<=p_max )
constraints.append( p@np.array([1,1,0,0,0]) <= p_g_1 )
constraints.append( p@np.array([0,0,1,1,1]) <= p_g_2 )
constraints.append( G@p <= p_rc )


# max ( min S_i/(I_i + sigma_i))
# (I_i+sigma)/S_i >= alpha
# minimize alpha
# answer 1/alpha
# 



In [17]:
def solver(constraints, sigma, l = 0, u = 10):
    alpha = cp.Parameter()
    alpha.value = (l+u)/2
    S = cp.multiply(G.diagonal(), p)
    I = G@p - S
    constraints.append( (I+sigma) <= alpha*S )
    objective = cp.Minimize(alpha)
    prob = cp.Problem(objective, constraints)
    if (u-l)<=0.005:
        return 1/alpha.value
    try:
        prob.solve()
        if prob.status == 'optimal':
            return solver(constraints[:-1], sigma, l = l, u = alpha.value)
        else:
            return solver(constraints[:-1], sigma, l = alpha.value, u = u)
    except:
        return solver(constraints[:-1], sigma, u =u*10)
        
print("the maximum SINR value is",solver(constraints, sigma))


the maximum SINR value is 1.68559670781893
